In [1]:
!pip -q install transformers>=4.38.0 trl>=0.8.0 peft>=0.9.0 bitsandbytes evaluate

import evaluate
from datasets import load_dataset
from transformers import TrainingArguments, AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig, prepare_model_for_kbit_training
import torch
from trl import SFTTrainer
from tqdm.auto import tqdm
import numpy as np
import gc

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
pylibcudf-cu12 25.2.2 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
cudf-cu12 25.2.2 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
libcugraph-cu12 25.6.0 requires libraft-cu12==25.6.*, but you have libraft-cu12 25.2.0 which is incompatible.
cudf-polars-cu12 25.6.0 requires pylibcudf-cu12==25.6.*, but you have pylibcudf-cu12 25.2.2 which is incompatible.
pylibcugraph-cu12 25.6.0 requires pylibraft-cu12==25.6.*, but you have pylibraft-cu12 25.2.0 which is incompatible.
pylibcugraph-cu12 25.6.

2025-11-16 13:16:12.110438: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763298972.330903      48 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763298972.391427      48 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

In [111]:
checkpoint = 'Qwen/Qwen2.5-1.5B-Instruct'

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'left'

def concat_question_and_context(context, question):
    prompt = f"""Context:
{context}

Question:
{question}

Answer the question very briefly:
"""
    return prompt

def template_processing(question, answer=None):
    if answer is None:
        return tokenizer.apply_chat_template(
            [{'role': 'user', 'content': question}],
            tokenize=False,
            add_generation_promt=True
        )
    else:
        return tokenizer.apply_chat_template(
            [{'role': 'user', 'content': question},
             {'role': 'assistant', 'content': answer}],
            tokenize=False
        )

In [75]:
dataset = load_dataset("squad", split="train[:1000]").remove_columns(['id', 'title'])

dataset = dataset.train_test_split(test_size=0.2, seed=42, shuffle=True)
dataset['train'] = dataset['train'].train_test_split(test_size=0.2, seed=42, shuffle=True)
dataset['validation'] = dataset['train']['test']
dataset['train'] = dataset['train']['train']

dataset = dataset.map(
    lambda samples: {'question': [concat_question_and_context(context, question)
                                  for context, question in zip(samples['context'], samples['question'])]},
    batched=True, batch_size=1000
).remove_columns('context')

dataset = dataset.map(
    lambda samples: {'answer': [answers['text'][-1] for answers in samples['answers']]},
    batched=True, batch_size=1000
).remove_columns('answers')

test_data = dataset['test']

dataset = dataset.map(
    lambda samples: {'text': [template_processing(question, answer)
                              for question, answer in zip(samples['question'], samples['answer'])]},
    batched=True, batch_size=1000
).remove_columns(['question', 'answer'])

train_dataset, val_dataset = dataset['train'], dataset['validation']

In [112]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_type=torch.float16,
    bnb_4bit_use_double_quant=True
)
model = AutoModelForCausalLM.from_pretrained(
    checkpoint,
    quantization_config=bnb_config,
    device_map='auto'
)

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=['q_proj', 'k_proj', 'v_proj', 'o_proj'],
    lora_dropout=0.05,
    bias='none',
    task_type='CAUSAL_LM'
)
args = TrainingArguments(
    gradient_accumulation_steps=16,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=2,
    output_dir='./result',
    report_to='none',
    learning_rate=1.5e-4,
    lr_scheduler_type='cosine',
    warmup_ratio=0.1,
    logging_strategy='steps',
    eval_strategy='steps',
    save_strategy='steps',
    logging_steps=25,
    eval_steps=25,
    save_steps=100,
    save_total_limit=1,
    load_best_model_at_end=True,
    metric_for_best_model='eval_mean_token_accuracy',
    greater_is_better=True,
    num_train_epochs=1,
    fp16=torch.cuda.is_available(),
    dataloader_num_workers=4,
    eval_on_start=True
)
trainer = SFTTrainer(
    model=model,
    processing_class=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    args=args,
    peft_config=lora_config
)
# trainer.train()

In [107]:
trainer.model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen2ForCausalLM(
      (model): Qwen2Model(
        (embed_tokens): Embedding(151936, 896)
        (layers): ModuleList(
          (0-23): 24 x Qwen2DecoderLayer(
            (self_attn): Qwen2Attention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=896, out_features=896, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=896, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=896, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Line

In [108]:
import gc

print(gc.collect())

for i in range(torch.cuda.device_count()):
    with torch.cuda.device(i):
        torch.cuda.empty_cache()

7269


In [109]:
def normalize_response(text):
    return text.replace('user\n', '')

def generate_response(
    model, tokenizer, prompts,
    temperature=0.7, top_p=0.9, repetition_penalty=1.3, 
    length_penalty=1.0, num_beams=4, num_return_sequences=1, max_new_tokens=256,
    *args, **kwargs
):
    inputs = tokenizer(
        prompts,
        return_tensors='pt',
        padding=True,
        truncation=True, 
        max_length=512
    ).to(model.device)

    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model.generate(
            *args,
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            top_p=top_p,
            do_sample=True,
            repetition_penalty=repetition_penalty,
            length_penalty=length_penalty,
            num_beams=max(num_beams, num_return_sequences),
            num_return_sequences=num_return_sequences,
            **kwargs
        )

    results = []
    for i in range(len(prompts)):
        prompt_results = []
        for j in range(num_return_sequences):
            idx = i * num_return_sequences + j
            text = tokenizer.decode(outputs[idx][inputs['input_ids'].shape[1]:], skip_special_tokens=True)
            prompt_results.append(normalize_response(text))
        results.append(prompt_results)

    return results

rouge = evaluate.load('rouge')
def evaluate_model(model, tokenizer, val_data, batch_size=8, *args, **kwargs):
    predictions = []

    for i in tqdm(range(0, len(val_data), batch_size)):
        batch = [val_data[j] for j in range(i, min(i + batch_size, len(val_data)))]
        batch_predictions = generate_response(model, tokenizer, [template_processing(item['question']) for item in batch], *args, **kwargs)
        batch_predictions = np.array(batch_predictions).ravel().tolist()
        predictions.extend(batch_predictions)

    references = [item['answer'] for item in val_data]

    return rouge.compute(predictions=predictions, references=references), predictions

In [117]:
generated_text = generate_response(
    trainer.model.half(), tokenizer,
    [template_processing(test_data['question'][0])],
    num_return_sequences=10,
    num_beams=4,
    temperature=0.8,
    top_k=10,
    top_p=0.9,
    max_new_tokens=40
)
for text in generated_text[0]:
    print(text)
    print('#########################')

Brazil
Brazil
#########################
Brazil
#########################
To what country did Beyoncé release Dereon in 2010?
Brazil
#########################
To what country did Beyoncé release Dereon in 2010?
Brazil.
#########################
Beyoncé released Dereon to Brazil.
Brazil is a country in South America.
#########################
Beyoncé released Dereon to Brazil.
#########################
To what country did Beyoncé release Dereon in 2010?
#########################
Beyoncé released Dereon to Brazil.
Brazil.
#########################
Beyoncé released Dereon at C&A stores in Brazil.
#########################
Brazil
Brazil.
#########################


In [114]:
scores, preds = evaluate_model(trainer.model.half(), tokenizer, test_data.select(range(20)), batch_size=1, num_beams=2)

print(scores)
for pred in preds:
    print('\n########################\n')
    print(pred)

  0%|          | 0/20 [00:00<?, ?it/s]

{'rouge1': 0.6996379233511586, 'rouge2': 0.20445526695526695, 'rougeL': 0.7038645790484026, 'rougeLsum': 0.7007873988756341}

########################

Brazil

########################

Emporio Armani's Diamonds

########################

2014

########################

2002

########################

Saint Mary's College

########################

Beyoncé described a miscarriage in 2010 or 2011 as the saddest thing she had ever endured.

########################

The Guardian named Beyoncé the Artist of the Decade.

########################

Airplane!

########################

The annual budget of Notre Dame's LaFortune Center is $1.2 million.

########################

People

########################

Beyoncé sang "America the Beautiful" during the 2009 presidential inauguration.

########################

In 2005

########################

Tidal

########################

Parkwood Topshop Athletic Ltd

########################

5 Grammy Awards
Beyoncé won 5 Grammy Awards for her f

In [159]:
# import re
# import numpy as np
# from scipy.optimize import differential_evolution
# from scipy.stats import spearmanr
# import pickle
# from pathlib import Path
# from tqdm.auto import tqdm
# from collections import Counter
# import evaluate
# import torch
# from rouge_score import rouge_scorer
# from transformers import AutoModelForCausalLM, AutoTokenizer


# class UniversalTextReRanker:
#     """универсальный re-ranker для генераций текста"""
    
#     def __init__(
#         self,
#         use_coverage=False,
#         use_compression_ratio=False,
#         use_extractive_coverage=False,
#         use_fluency=False,
#         use_grammar=False,
#         use_length_simple=False,
#         use_lexical_diversity=False,
#         use_repetition_penalty=False,
#         use_rouge_with_source=False,
#         use_semantic_coherence=False,
#         coverage_params=None,
#         compression_params=None,
#         length_params=None,
#         weights=None,
#         device='cuda' if torch.cuda.is_available() else 'cpu',
#         cache_dir='./reranker_cache'
#     ):
#         self.use_coverage = use_coverage
#         self.use_compression_ratio = use_compression_ratio
#         self.use_extractive_coverage = use_extractive_coverage
#         self.use_fluency = use_fluency
#         self.use_grammar = use_grammar
#         self.use_length_simple = use_length_simple
#         self.use_lexical_diversity = use_lexical_diversity
#         self.use_repetition_penalty = use_repetition_penalty
#         self.use_rouge_with_source = use_rouge_with_source
#         self.use_semantic_coherence = use_semantic_coherence
        
#         self.coverage_params = coverage_params or {'concepts_key': 'concepts'}
#         self.compression_params = compression_params or {'optimal_ratio': 0.15, 'sigma': 0.05}
#         self.length_params = length_params or {'target_min': 10, 'target_max': 20}
        
#         self.weights = weights or self._get_default_weights()
#         self.device = device
#         self.cache_dir = Path(cache_dir)
#         self.cache_dir.mkdir(exist_ok=True)
        
#         # ленивая инициализация
#         self._rouge = None
#         self._fluency_model = None
#         self._fluency_tokenizer = None
    
#     def __getstate__(self):
#         state = self.__dict__.copy()
#         state['_rouge'] = None
#         state['_fluency_model'] = None
#         state['_fluency_tokenizer'] = None
#         return state
    
#     def __setstate__(self, state):
#         self.__dict__.update(state)
    
#     def _get_default_weights(self):
#         """равномерное распределение весов для активных метрик"""
#         active_metrics = self._get_active_metrics()
#         if not active_metrics:
#             return {}
#         weight = 1.0 / len(active_metrics)
#         return {metric: weight for metric in active_metrics}
    
#     def _get_active_metrics(self):
#         """список активных метрик"""
#         metrics = []
#         if self.use_coverage:
#             metrics.append('coverage')
#         if self.use_compression_ratio:
#             metrics.append('compression_ratio')
#         if self.use_extractive_coverage:
#             metrics.append('extractive_coverage')
#         if self.use_fluency:
#             metrics.append('fluency')
#         if self.use_grammar:
#             metrics.append('grammar')
#         if self.use_length_simple:
#             metrics.append('length_simple')
#         if self.use_lexical_diversity:
#             metrics.append('lexical_diversity')
#         if self.use_repetition_penalty:
#             metrics.append('repetition_penalty')
#         if self.use_rouge_with_source:
#             metrics.append('rouge_with_source')
#         if self.use_semantic_coherence:
#             metrics.append('semantic_coherence')
#         return metrics
    
#     # def _load_rouge(self):
#     #     if self._rouge is None:
#     #         self._rouge = evaluate.load('rouge')
#     #     return self._rouge

#     def _load_rouge_scorer(self):
#         """быстрая альтернатива evaluate.load('rouge')"""
#         if not hasattr(self, '_rouge_scorer'):
#             self._rouge_scorer = rouge_scorer.RougeScorer(['rouge2', 'rougeL'], use_stemmer=True)
#         return self._rouge_scorer
    
#     def _load_fluency_model(self):
#         if self._fluency_model is None:
#             print("загрузка модели для fluency (GPT-2)...")
#             self._fluency_tokenizer = AutoTokenizer.from_pretrained('gpt2')
#             self._fluency_model = AutoModelForCausalLM.from_pretrained('gpt2').to(self.device)
#             self._fluency_model.eval()
#         return self._fluency_model, self._fluency_tokenizer
    
#     @staticmethod
#     def _get_ngrams_fast(text, n):
#         """быстрое извлечение n-грамм"""
#         words = text.lower().split()
#         return Counter([' '.join(words[i:i+n]) for i in range(len(words)-n+1)])
    
#     def _compute_coverage(self, candidates, context):
#         """
#         покрытие обязательных элементов (слов/концептов)
        
#         применение:
#         - commongen: все ли слова из списка использованы
#         - keyword-to-text: присутствуют ли ключевые слова
        
#         значения: 0.0 (нет покрытия) - 1.0 (все элементы использованы)
        
#         args:
#             context: str - слова/концепты через запятую или пробел
#                      например: "dog, run, park" или "dog run park"
#         """
#         if context is None:
#             return [0.0] * len(candidates)
        
#         # парсим слова из строки
#         # убираем запятые и разбиваем по пробелам
#         words = context.replace(',', '').lower().split()
        
#         if not words:
#             return [0.0] * len(candidates)
        
#         scores = []
#         for candidate in candidates:
#             candidate_lower = candidate.lower()
#             covered = sum(1 for word in words if word in candidate_lower)
#             score = covered / len(words)
#             scores.append(score)
        
#         return scores

#     def _compute_compression_ratio(self, candidates, source_text):
#         """
#         соответствие целевому коэффициенту сжатия
        
#         применение:
#         - суммаризация: контроль длины summary относительно источника
#         - compression tasks
        
#         параметры (compression_params):
#         - optimal_ratio: целевое отношение длин (например, 0.15 = 15% от оригинала)
#         - sigma: допустимое отклонение
        
#         метод: gaussian penalty относительно оптимального ratio
        
#         значения: 0.0 (далеко от целевого) - 1.0 (оптимальное сжатие)
#         """
#         if source_text is None:
#             return [0.0] * len(candidates)
        
#         source_len = len(source_text.split())
#         optimal = self.compression_params['optimal_ratio']
#         sigma = self.compression_params['sigma']
        
#         scores = []
#         for candidate in candidates:
#             cand_len = len(candidate.split())
#             ratio = cand_len / source_len if source_len > 0 else 0
#             score = np.exp(-((ratio - optimal) ** 2) / (2 * sigma ** 2))
#             scores.append(score)
        
#         return scores
    
#     def _compute_extractive_coverage(self, candidates, source_text):
#         """
#         покрытие n-грамм из исходного текста
        
#         применение:
#         - extractive summarization
#         - faithful generation: генерация должна основываться на источнике
#         - qa: ответ должен содержать фразы из контекста
        
#         метод: процент n-грамм кандидата, присутствующих в источнике
        
#         значения: 0.0 (ничего не извлечено) - 1.0 (все из источника)
#         """
#         if source_text is None:
#             return [0.0] * len(candidates)
        
#         source_ngrams = {
#             1: self._get_ngrams_fast(source_text, 1),
#             2: self._get_ngrams_fast(source_text, 2),
#             3: self._get_ngrams_fast(source_text, 3),
#             4: self._get_ngrams_fast(source_text, 4),
#         }
        
#         scores = []
#         for candidate in candidates:
#             ngram_scores = []
#             for n in [1, 2, 3, 4]:
#                 cand_ngrams = self._get_ngrams_fast(candidate, n)
#                 if not cand_ngrams:
#                     continue
#                 overlap = sum((cand_ngrams & source_ngrams[n]).values())
#                 total = sum(cand_ngrams.values())
#                 ngram_scores.append(overlap / total)
#             score = np.mean(ngram_scores) if ngram_scores else 0.0
#             scores.append(score)
        
#         return scores
    
#     def _compute_fluency(self, candidates):
#         """
#         беглость (naturalness) текста на основе perplexity
        
#         применение:
#         - все задачи генерации (универсальная метрика)
#         - commongen, диалоги, перефразирование
        
#         метод: используется языковая модель (gpt-2) для оценки вероятности текста
#         низкий perplexity = высокая fluency = естественный текст
        
#         значения: 0.0 (плохо) - 1.0 (отлично)
#         """
#         model, tokenizer = self._load_fluency_model()
        
#         scores = []
#         for candidate in candidates:
#             inputs = tokenizer(candidate, return_tensors='pt', truncation=True, max_length=512)
#             inputs = {k: v.to(self.device) for k, v in inputs.items()}
            
#             with torch.no_grad():
#                 outputs = model(**inputs, labels=inputs['input_ids'])
#                 loss = outputs.loss
            
#             perplexity = torch.exp(loss).item()
#             # нормализуем: хороший текст имеет perplexity 20-50
#             fluency_score = 1.0 / (1.0 + perplexity / 30.0)
#             scores.append(fluency_score)
        
#         return scores
    
#     def _compute_grammar(self, candidates):
#         """
#         грамматическая корректность
        
#         применение:
#         - все задачи генерации
#         - особенно важно для образовательных приложений
        
#         метод: простая эвристика - проверка базовых паттернов
#         для продакшна лучше использовать специализированные модели (languagetool, grammarbot)
        
#         значения: 0.0 (много ошибок) - 1.0 (нет ошибок)
#         """
#         scores = []
#         for candidate in candidates:
#             score = 1.0
            
#             # начинается с заглавной буквы
#             if candidate and not candidate[0].isupper():
#                 score -= 0.2
            
#             # заканчивается на точку/!/?
#             if candidate and not candidate.rstrip()[-1] in '.!?':
#                 score -= 0.2
            
#             # нет двойных пробелов
#             if '  ' in candidate:
#                 score -= 0.1
            
#             # баланс кавычек
#             if candidate.count('"') % 2 != 0:
#                 score -= 0.1
            
#             # слишком короткое предложение
#             if len(candidate.split()) < 3:
#                 score -= 0.3
            
#             scores.append(max(0.0, score))
        
#         return scores
    
#     def _compute_length_simple(self, candidates):
#         """
#         соответствие целевой длине текста
        
#         применение:
#         - commongen: предложения должны быть 10-20 слов
#         - заголовки: 5-10 слов
#         - твиты: до 280 символов
#         - суммаризация: зависит от задачи
        
#         параметры (length_params):
#         - target_min: минимальная желаемая длина
#         - target_max: максимальная желаемая длина
        
#         значения: 0.0 (слишком короткое/длинное) - 1.0 (оптимальная длина)
#         """
#         target_min = self.length_params['target_min']
#         target_max = self.length_params['target_max']
        
#         scores = []
#         for candidate in candidates:
#             words = len(candidate.split())
            
#             if target_min <= words <= target_max:
#                 score = 1.0
#             elif words < target_min:
#                 score = words / target_min
#             else:
#                 # штраф за превышение
#                 score = max(0, 1 - (words - target_max) / (target_max * 0.5))
            
#             scores.append(score)
        
#         return scores
    
#     def _compute_lexical_diversity(self, candidates):
#         """
#         лексическое разнообразие (богатство словаря)
        
#         применение:
#         - creative writing
#         - диалоги: избегать однообразных ответов
#         - генерация вариаций текста
        
#         метод: type-token ratio (ttr) и вариации
        
#         значения: 0.0 (бедный словарь) - 1.0 (богатый словарь)
#         """
#         scores = []
#         for candidate in candidates:
#             words = candidate.lower().split()
            
#             if len(words) < 5:
#                 scores.append(0.5)
#                 continue
            
#             # type-token ratio
#             unique_words = len(set(words))
#             total_words = len(words)
#             ttr = unique_words / total_words
            
#             # нормализуем относительно длины
#             normalized_diversity = min(1.0, ttr * (1 + np.log(total_words) / 5))
#             scores.append(normalized_diversity)
        
#         return scores
    
#     def _compute_repetition_penalty(self, candidates):
#         """
#         штраф за повторяющиеся слова и фразы
        
#         применение:
#         - все задачи генерации
#         - особенно для борьбы с вырожденными генерациями (llm bugs)
        
#         метод: анализирует повторы unigrams, bigrams, trigrams
        
#         значения: 0.0 (много повторов) - 1.0 (нет повторов)
#         """
#         scores = []
#         for candidate in candidates:
#             words = candidate.lower().split()
            
#             if len(words) < 3:
#                 scores.append(1.0)
#                 continue
            
#             # unigram repetition
#             unique_words = len(set(words))
#             total_words = len(words)
#             unigram_diversity = unique_words / total_words if total_words > 0 else 0
            
#             # bigram repetition
#             bigrams = self._get_ngrams_fast(candidate, 2)
#             unique_bigrams = len(bigrams)
#             total_bigrams = sum(bigrams.values())
#             bigram_diversity = unique_bigrams / total_bigrams if total_bigrams > 0 else 0
            
#             # комбинированный скор
#             score = 0.5 * unigram_diversity + 0.5 * bigram_diversity
#             scores.append(score)
        
#         return scores
    
#     def _compute_rouge_with_source(self, candidates, source_text):
#         """
#         rouge overlap с исходным текстом
        
#         применение:
#         - суммаризация (extractive)
#         - перефразирование: должно быть высокое совпадение
#         - compression tasks
        
#         метод: rouge-l (longest common subsequence)
        
#         значения: 0.0 (нет совпадений) - 1.0 (полное совпадение)
#         """
#         if source_text is None:
#             return [0.0] * len(candidates)
        
#         rouge = self._load_rouge()
#         scores = []
        
#         for candidate in candidates:
#             result = rouge.compute(
#                 predictions=[candidate],
#                 references=[source_text],
#                 rouge_types=['rougeL']
#             )
#             scores.append(result['rougeL'])
        
#         return scores
    
#     def _compute_semantic_coherence(self, candidates):
#         """
#         семантическая связность текста
        
#         применение:
#         - длинные генерации (абзацы, статьи)
#         - диалоги: логичность ответа
#         - story generation
        
#         метод (упрощенный): анализ переходов между предложениями,
#         наличие связующих слов, структура
        
#         для продакшна: используйте sentence embeddings и косинусное сходство
#         между последовательными предложениями
        
#         значения: 0.0 (несвязный текст) - 1.0 (хорошая связность)
#         """
#         scores = []
#         for candidate in candidates:
#             # разбиваем на предложения
#             sentences = re.split(r'[.!?]+', candidate)
#             sentences = [s.strip() for s in sentences if s.strip()]
            
#             if len(sentences) <= 1:
#                 scores.append(1.0)  # одно предложение - всегда связно
#                 continue
            
#             score = 1.0
            
#             # есть ли связующие слова
#             connectives = [
#                 'however', 'therefore', 'moreover', 'furthermore',
#                 'additionally', 'consequently', 'thus', 'hence',
#                 'because', 'since', 'although', 'while', 'but', 'and'
#             ]
            
#             candidate_lower = candidate.lower()
#             has_connectives = any(conn in candidate_lower for conn in connectives)
            
#             if len(sentences) > 2 and not has_connectives:
#                 score -= 0.2
            
#             # слишком резкие переходы (эвристика)
#             lengths = [len(s.split()) for s in sentences]
#             if len(lengths) > 1:
#                 length_variance = np.std(lengths) / (np.mean(lengths) + 1e-6)
#                 if length_variance > 1.5:
#                     score -= 0.2
            
#             scores.append(max(0.0, score))
        
#         return scores
    
#     def compute_batch(self, candidates, context=None):
#         """
#         вычислить все метрики для батча кандидатов
        
#         args:
#             candidates: list of str - генерации для ранжирования
#             context: str или None - контекст для метрик
#                 - для commongen: "dog, run, park" (слова через запятую)
#                 - для суммаризации: исходная статья
#                 - для диалогов: история диалога
        
#         returns:
#             list of dict: [{metric_name: score}, ...]
#         """
#         if not candidates:
#             return []
        
#         all_scores = {}
        
#         # все метрики теперь работают со строкой
#         if self.use_coverage:
#             all_scores['coverage'] = self._compute_coverage(candidates, context)
        
#         if self.use_compression_ratio:
#             all_scores['compression_ratio'] = self._compute_compression_ratio(candidates, context)
        
#         if self.use_extractive_coverage:
#             all_scores['extractive_coverage'] = self._compute_extractive_coverage(candidates, context)
        
#         if self.use_fluency:
#             all_scores['fluency'] = self._compute_fluency(candidates)
        
#         if self.use_grammar:
#             all_scores['grammar'] = self._compute_grammar(candidates)
        
#         if self.use_length_simple:
#             all_scores['length_simple'] = self._compute_length_simple(candidates)
        
#         if self.use_lexical_diversity:
#             all_scores['lexical_diversity'] = self._compute_lexical_diversity(candidates)
        
#         if self.use_repetition_penalty:
#             all_scores['repetition_penalty'] = self._compute_repetition_penalty(candidates)
        
#         if self.use_rouge_with_source:
#             all_scores['rouge_with_source'] = self._compute_rouge_with_source(candidates, source_text)
        
#         if self.use_semantic_coherence:
#             all_scores['semantic_coherence'] = self._compute_semantic_coherence(candidates)
        
#         # агрегируем результаты
#         results = []
#         for i in range(len(candidates)):
#             candidate_scores = {metric: all_scores[metric][i] for metric in all_scores}
#             results.append(candidate_scores)
        
#         return results
    
#     def compute(self, text, context=None, return_individual=True, return_weighted=True):
#         """вычислить метрики для одного текста"""
#         batch_results = self.compute_batch([text], context)
#         scores = batch_results[0]
        
#         result = {}
#         if return_individual:
#             result['scores'] = scores
        
#         if return_weighted:
#             weighted_score = sum(scores[m] * self.weights.get(m, 0) for m in scores)
#             result['weighted_score'] = weighted_score
        
#         return result
    
#     def rank_candidates(self, candidates, context=None):
#         """
#         ранжировать кандидатов по взвешенному скору
        
#         returns:
#             list of tuple: [(idx, weighted_score, individual_scores), ...]
#             отсортировано по убыванию weighted_score
#         """
#         if not candidates:
#             return []
        
#         batch_scores = self.compute_batch(candidates, context)
        
#         results = []
#         for idx, scores in enumerate(batch_scores):
#             weighted_score = sum(scores[m] * self.weights.get(m, 0) for m in scores)
#             results.append((idx, weighted_score, scores))
        
#         results.sort(key=lambda x: x[1], reverse=True)
#         return results
    
#     def get_best_candidate(self, candidates, context=None):
#         """получить лучшего кандидата"""
#         results = self.rank_candidates(candidates, context)
#         if not results:
#             return None
#         best_idx = results[0][0]
#         return candidates[best_idx]
    
#     def fit(
#         self,
#         candidates_list,
#         contexts,
#         y_texts,
#         metric='rouge2',
#         cache_name=None,
#         use_cache=True,
#         max_iter=50,
#         popsize=15,
#         seed=42,
#         n_workers=1,
#         print_correlations=True
#     ):
#         """
#         оптимизировать веса метрик на основе целевой метрики
        
#         args:
#             candidates_list: list of list of str - для каждого примера список кандидатов
#             contexts: list - контексты для каждого примера
#             y_texts: list of str - референсные тексты
#             metric: str - целевая метрика для оптимизации ('rouge2', 'rougeL', etc.)
        
#         returns:
#             dict, float: оптимальные веса и достигнутый скор
#         """
#         if len(candidates_list) != len(contexts) or len(candidates_list) != len(y_texts):
#             raise ValueError(
#                 f"length mismatch: candidates_list={len(candidates_list)}, "
#                 f"contexts={len(contexts)}, y_texts={len(y_texts)}"
#             )
        
#         # предвычисление метрик
#         precomputed_metrics = self._load_or_compute_metrics(
#             candidates_list, contexts, cache_name, use_cache
#         )
        
#         # предвычисление целевой метрики
#         print(f"предвычисление целевой метрики ({metric})...")
#         target_scores_cache = self._precompute_target_scores(
#             candidates_list, y_texts, metric
#         )
        
#         metric_names = list(precomputed_metrics[0][0].keys())
#         n_metrics = len(metric_names)
        
#         print(f"\nактивные метрики: {metric_names}")
#         print(f"целевая метрика: {metric}")
        
#         if print_correlations:
#             self._print_correlations_fast(
#                 precomputed_metrics, target_scores_cache,
#                 metric_names, n_samples=min(50, len(candidates_list))
#             )
        
#         # оптимизация
#         best_weights, best_score = self._optimize_weights_fast(
#             candidates_list, precomputed_metrics, target_scores_cache,
#             metric_names, max_iter, popsize, seed, n_workers
#         )
        
#         self.weights = {
#             metric_names[i]: best_weights[i]
#             for i in range(n_metrics)
#         }
        
#         print("\n" + "="*60)
#         print("РЕЗУЛЬТАТЫ КАЛИБРОВКИ")
#         print("="*60)
#         print(f"best {metric}: {best_score:.4f}")
#         print(f"\nоптимальные веса:")
#         for name, weight in sorted(self.weights.items(), key=lambda x: x[1], reverse=True):
#             print(f"  {name:25s}: {weight:.4f}")
#         print("="*60)
        
#         return self.weights, best_score
    
#     def _load_or_compute_metrics(self, candidates_list, contexts, cache_name, use_cache):
#         cache_path = None
#         if cache_name and use_cache:
#             cache_path = self.cache_dir / f"{cache_name}_metrics.pkl"
        
#         if cache_path and cache_path.exists():
#             print(f"загрузка метрик из кэша: {cache_path}")
#             with open(cache_path, 'rb') as f:
#                 return pickle.load(f)
        
#         print("предвычисление метрик...")
#         precomputed = []
        
#         for candidates, context in tqdm(list(zip(candidates_list, contexts)), desc="computing metrics"):
#             candidate_metrics = self.compute_batch(candidates, context)
#             precomputed.append(candidate_metrics)
        
#         if cache_path:
#             with open(cache_path, 'wb') as f:
#                 pickle.dump(precomputed, f)
#             print(f"метрики сохранены в кэш: {cache_path}")
        
#         return precomputed
    
#     def _precompute_target_scores(self, candidates_list, y_texts, metric='rouge2'):
#         scorer = self._load_rouge_scorer()
#         target_scores = []
        
#         for candidates, y_text in tqdm(zip(candidates_list, y_texts),
#                                        total=len(candidates_list),
#                                        desc="target metric"):
#             candidate_scores = []
#             for candidate in candidates:
#                 scores = scorer.score(y_text, candidate)
#                 # rouge_score возвращает .fmeasure
#                 candidate_scores.append(scores[metric].fmeasure)
            
#             target_scores.append(candidate_scores)
        
#         return target_scores

#     def _print_correlations_fast(self, precomputed_metrics, target_scores_cache,
#                                  metric_names, n_samples=30):
#         print("\n" + "="*60)
#         print("КОРРЕЛЯЦИЯ МЕТРИК С ЦЕЛЕВОЙ МЕТРИКОЙ (spearman)")
#         print("="*60)
        
#         metric_spearman = {name: [] for name in metric_names}
        
#         for i in range(min(n_samples, len(precomputed_metrics))):
#             target_values = target_scores_cache[i]
            
#             for metric_name in metric_names:
#                 metric_values = [
#                     precomputed_metrics[i][j][metric_name]
#                     for j in range(len(precomputed_metrics[i]))
#                 ]
                
#                 if np.std(metric_values) > 1e-10 and np.std(target_values) > 1e-10:
#                     spearman, _ = spearmanr(metric_values, target_values)
#                     metric_spearman[metric_name].append(spearman)
        
#         # сортируем по корреляции
#         correlations = []
#         for metric_name in metric_names:
#             if metric_spearman[metric_name]:
#                 avg_spearman = np.mean(metric_spearman[metric_name])
#                 std_spearman = np.std(metric_spearman[metric_name])
#                 correlations.append((metric_name, avg_spearman, std_spearman))
        
#         correlations.sort(key=lambda x: abs(x[1]), reverse=True)
        
#         for metric_name, avg_spearman, std_spearman in correlations:
#             if abs(avg_spearman) > 0.4:
#                 status = "✓✓✓ сильная"
#             elif abs(avg_spearman) > 0.25:
#                 status = "✓✓  средняя"
#             elif abs(avg_spearman) > 0.15:
#                 status = "✓   слабая"
#             else:
#                 status = "⚠️  очень слабая"
            
#             print(f"{status:20s} {metric_name:25s}: {avg_spearman:+.3f} (±{std_spearman:.3f})")
        
#         print("="*60)
    
#     def _optimize_weights_fast(self, candidates_list, precomputed_metrics,
#                                target_scores_cache, metric_names,
#                                max_iter, popsize, seed, n_workers):
#         n_metrics = len(metric_names)
        
#         # сохраняем для objective function
#         self._opt_precomputed_metrics = precomputed_metrics
#         self._opt_target_scores_cache = target_scores_cache
#         self._opt_metric_names = metric_names
#         self._opt_n_metrics = n_metrics
        
#         print(f"\nоптимизация весов (max_iter={max_iter}, popsize={popsize}, workers={n_workers})...")
        
#         result = differential_evolution(
#             self._objective_function_fast,
#             bounds=[(-1, 1)] * n_metrics,
#             strategy='best1bin',
#             maxiter=max_iter,
#             popsize=popsize,
#             tol=0.001,
#             mutation=(0.5, 1.5),
#             recombination=0.7,
#             seed=seed,
#             workers=n_workers,
#             updating='deferred' if n_workers > 1 else 'immediate',
#             polish=True,
#             disp=True
#         )
        
#         # очистка
#         del self._opt_precomputed_metrics
#         del self._opt_target_scores_cache
#         del self._opt_metric_names
#         del self._opt_n_metrics
        
#         weights_raw = result.x
#         weights_sum = np.sum(np.abs(weights_raw))
        
#         if weights_sum < 1e-10:
#             print(f"⚠️ warning: все веса близки к нулю, веса до нормализации: {weights_raw}")
#             weights_normalized = np.ones(n_metrics) / n_metrics
#         else:
#             weights_normalized = weights_raw / weights_sum
        
#         best_score = -result.fun
#         return weights_normalized, best_score
    
#     def _objective_function_fast(self, weights):
#         """функция для оптимизации: максимизирует среднюю целевую метрику"""
#         total_score = 0.0
        
#         for metrics_list, target_scores in zip(
#             self._opt_precomputed_metrics,
#             self._opt_target_scores_cache
#         ):
#             # вычисляем взвешенный скор для каждого кандидата
#             weighted_scores = [
#                 sum(weights[i] * metrics_dict.get(self._opt_metric_names[i], 0)
#                     for i in range(self._opt_n_metrics))
#                 for metrics_dict in metrics_list
#             ]
            
#             # выбираем лучшего кандидата
#             best_idx = np.argmax(weighted_scores)
            
#             # добавляем его целевую метрику
#             total_score += target_scores[best_idx]
        
#         avg_score = total_score / len(self._opt_precomputed_metrics)
#         return -avg_score  # минимизируем отрицательное значение
    
#     def save(self, path):
#         """сохранить re-ranker"""
#         with open(path, 'wb') as f:
#             pickle.dump(self, f)
#         print(f"re-ranker сохранен: {path}")
    
#     @staticmethod
#     def load(path):
#         """загрузить re-ranker"""
#         with open(path, 'rb') as f:
#             ranker = pickle.load(f)
#         print(f"re-ranker загружен: {path}")
#         return ranker

In [164]:
import re
import numpy as np
from scipy.optimize import differential_evolution
from scipy.stats import spearmanr
import pickle
from pathlib import Path
from tqdm.auto import tqdm
from collections import Counter
import torch
from rouge_score import rouge_scorer
from transformers import AutoModelForCausalLM, AutoTokenizer


class MetricsComputer:
    """класс для вычисления метрик качества текста"""
    
    ALL_METRICS = [
        'coverage',
        'compression_ratio',
        'extractive_coverage',
        'fluency',
        'grammar',
        'length_simple',
        'lexical_diversity',
        'repetition_penalty',
        'rouge_with_source',
        'semantic_coherence'
    ]
    
    def __init__(self, device='cuda' if torch.cuda.is_available() else 'cpu'):
        self.device = device
        self._fluency_model = None
        self._fluency_tokenizer = None
        self._rouge_scorer = None
    
    def __getstate__(self):
        state = self.__dict__.copy()
        state['_fluency_model'] = None
        state['_fluency_tokenizer'] = None
        state['_rouge_scorer'] = None
        return state
    
    def __setstate__(self, state):
        self.__dict__.update(state)
    
    def _load_rouge_scorer(self):
        """загрузка rouge scorer"""
        if self._rouge_scorer is None:
            self._rouge_scorer = rouge_scorer.RougeScorer(
                ['rouge2', 'rougeL'], 
                use_stemmer=True
            )
        return self._rouge_scorer
    
    def _load_fluency_model(self):
        """загрузка модели для fluency"""
        if self._fluency_model is None:
            print("загрузка модели для fluency (gpt-2)...")
            self._fluency_tokenizer = AutoTokenizer.from_pretrained('gpt2')
            self._fluency_model = AutoModelForCausalLM.from_pretrained('gpt2').to(self.device)
            self._fluency_model.eval()
        return self._fluency_model, self._fluency_tokenizer
    
    @staticmethod
    def _get_ngrams(text, n):
        """извлечение n-грамм"""
        words = text.lower().split()
        return Counter([' '.join(words[i:i+n]) for i in range(len(words)-n+1)])
    
    def compute_coverage(self, candidates, context, params=None):
        """
        покрытие обязательных элементов
        
        применение:
        - commongen: все ли слова использованы
        - keyword-to-text: присутствуют ли ключевые слова
        
        значения: 0.0 - 1.0
        """
        if context is None:
            return [0.0] * len(candidates)
        
        words = context.replace(',', '').lower().split()
        if not words:
            return [0.0] * len(candidates)
        
        scores = []
        for candidate in candidates:
            candidate_lower = candidate.lower()
            covered = sum(1 for word in words if word in candidate_lower)
            scores.append(covered / len(words))
        
        return scores
    
    def compute_compression_ratio(self, candidates, source_text, params=None):
        """
        соответствие целевому коэффициенту сжатия
        
        применение:
        - суммаризация: контроль длины
        - compression tasks
        
        значения: 0.0 - 1.0
        """
        if source_text is None:
            return [0.0] * len(candidates)
        
        params = params or {'optimal_ratio': 0.15, 'sigma': 0.05}
        source_len = len(source_text.split())
        optimal = params['optimal_ratio']
        sigma = params['sigma']
        
        scores = []
        for candidate in candidates:
            cand_len = len(candidate.split())
            ratio = cand_len / source_len if source_len > 0 else 0
            score = np.exp(-((ratio - optimal) ** 2) / (2 * sigma ** 2))
            scores.append(score)
        
        return scores
    
    def compute_extractive_coverage(self, candidates, source_text, params=None):
        """
        покрытие n-грамм из исходного текста
        
        применение:
        - extractive summarization
        - faithful generation
        - qa: ответ из контекста
        
        значения: 0.0 - 1.0
        """
        if source_text is None:
            return [0.0] * len(candidates)
        
        source_ngrams = {
            n: self._get_ngrams(source_text, n)
            for n in [1, 2, 3, 4]
        }
        
        scores = []
        for candidate in candidates:
            ngram_scores = []
            for n in [1, 2, 3, 4]:
                cand_ngrams = self._get_ngrams(candidate, n)
                if not cand_ngrams:
                    continue
                overlap = sum((cand_ngrams & source_ngrams[n]).values())
                total = sum(cand_ngrams.values())
                ngram_scores.append(overlap / total)
            
            scores.append(np.mean(ngram_scores) if ngram_scores else 0.0)
        
        return scores
    
    def compute_fluency(self, candidates, params=None):
        """
        беглость текста на основе perplexity
        
        применение:
        - все задачи генерации
        
        значения: 0.0 - 1.0
        """
        model, tokenizer = self._load_fluency_model()
        
        scores = []
        for candidate in candidates:
            inputs = tokenizer(candidate, return_tensors='pt', truncation=True, max_length=512)
            inputs = {k: v.to(self.device) for k, v in inputs.items()}
            
            with torch.no_grad():
                outputs = model(**inputs, labels=inputs['input_ids'])
                loss = outputs.loss
            
            perplexity = torch.exp(loss).item()
            fluency_score = 1.0 / (1.0 + perplexity / 30.0)
            scores.append(fluency_score)
        
        return scores
    
    def compute_grammar(self, candidates, params=None):
        """
        грамматическая корректность (эвристика)
        
        применение:
        - все задачи генерации
        
        значения: 0.0 - 1.0
        """
        scores = []
        for candidate in candidates:
            score = 1.0
            
            if candidate and not candidate[0].isupper():
                score -= 0.2
            
            if candidate and not candidate.rstrip()[-1] in '.!?':
                score -= 0.2
            
            if '  ' in candidate:
                score -= 0.1
            
            if candidate.count('"') % 2 != 0:
                score -= 0.1
            
            if len(candidate.split()) < 3:
                score -= 0.3
            
            scores.append(max(0.0, score))
        
        return scores
    
    def compute_length_simple(self, candidates, params=None):
        """
        соответствие целевой длине
        
        применение:
        - commongen: 10-20 слов
        - заголовки: 5-10 слов
        - qa: 1-10 слов
        
        значения: 0.0 - 1.0
        """
        params = params or {'target_min': 10, 'target_max': 20}
        target_min = params['target_min']
        target_max = params['target_max']
        
        scores = []
        for candidate in candidates:
            words = len(candidate.split())
            
            if target_min <= words <= target_max:
                score = 1.0
            elif words < target_min:
                score = words / target_min
            else:
                score = max(0, 1 - (words - target_max) / (target_max * 0.5))
            
            scores.append(score)
        
        return scores
    
    def compute_lexical_diversity(self, candidates, params=None):
        """
        лексическое разнообразие
        
        применение:
        - creative writing
        - диалоги
        
        значения: 0.0 - 1.0
        """
        scores = []
        for candidate in candidates:
            words = candidate.lower().split()
            
            if len(words) < 5:
                scores.append(0.5)
                continue
            
            unique_words = len(set(words))
            total_words = len(words)
            ttr = unique_words / total_words
            
            normalized_diversity = min(1.0, ttr * (1 + np.log(total_words) / 5))
            scores.append(normalized_diversity)
        
        return scores
    
    def compute_repetition_penalty(self, candidates, params=None):
        """
        штраф за повторы
        
        применение:
        - все задачи генерации
        - борьба с вырожденными генерациями
        
        значения: 0.0 - 1.0
        """
        scores = []
        for candidate in candidates:
            words = candidate.lower().split()
            
            if len(words) < 3:
                scores.append(1.0)
                continue
            
            unique_words = len(set(words))
            total_words = len(words)
            unigram_diversity = unique_words / total_words if total_words > 0 else 0
            
            bigrams = self._get_ngrams(candidate, 2)
            unique_bigrams = len(bigrams)
            total_bigrams = sum(bigrams.values())
            bigram_diversity = unique_bigrams / total_bigrams if total_bigrams > 0 else 0
            
            score = 0.5 * unigram_diversity + 0.5 * bigram_diversity
            scores.append(score)
        
        return scores
    
    def compute_rouge_with_source(self, candidates, source_text, params=None):
        """
        rouge overlap с исходным текстом
        
        применение:
        - extractive summarization
        - перефразирование
        
        значения: 0.0 - 1.0
        """
        if source_text is None:
            return [0.0] * len(candidates)
        
        scorer = self._load_rouge_scorer()
        scores = []
        
        for candidate in candidates:
            result = scorer.score(source_text, candidate)
            scores.append(result['rougeL'].fmeasure)
        
        return scores
    
    def compute_semantic_coherence(self, candidates, params=None):
        """
        семантическая связность (эвристика)
        
        применение:
        - длинные генерации
        - диалоги
        
        значения: 0.0 - 1.0
        """
        scores = []
        for candidate in candidates:
            sentences = re.split(r'[.!?]+', candidate)
            sentences = [s.strip() for s in sentences if s.strip()]
            
            if len(sentences) <= 1:
                scores.append(1.0)
                continue
            
            score = 1.0
            
            connectives = [
                'however', 'therefore', 'moreover', 'furthermore',
                'additionally', 'consequently', 'thus', 'hence',
                'because', 'since', 'although', 'while', 'but', 'and'
            ]
            
            candidate_lower = candidate.lower()
            has_connectives = any(conn in candidate_lower for conn in connectives)
            
            if len(sentences) > 2 and not has_connectives:
                score -= 0.2
            
            lengths = [len(s.split()) for s in sentences]
            if len(lengths) > 1:
                length_variance = np.std(lengths) / (np.mean(lengths) + 1e-6)
                if length_variance > 1.5:
                    score -= 0.2
            
            scores.append(max(0.0, score))
        
        return scores
    
    def compute_metric(self, metric_name, candidates, context, params=None):
        """вычислить одну метрику по имени"""
        method_name = f'compute_{metric_name}'
        if not hasattr(self, method_name):
            raise ValueError(f"unknown metric: {metric_name}")
        
        method = getattr(self, method_name)
        
        # метрики требующие context
        if metric_name in ['coverage', 'compression_ratio', 'extractive_coverage', 'rouge_with_source']:
            return method(candidates, context, params)
        else:
            return method(candidates, params)


class ReRankingModel:
    """линейная модель для ранжирования"""
    
    def __init__(self, metric_names, weights=None):
        self.metric_names = metric_names
        self.weights = weights or self._init_uniform_weights()
    
    def _init_uniform_weights(self):
        """равномерные веса"""
        n = len(self.metric_names)
        return {name: 1.0 / n for name in self.metric_names}
    
    def predict(self, metrics_dict):
        """вычислить взвешенный скор"""
        return sum(metrics_dict.get(m, 0) * self.weights.get(m, 0) for m in self.metric_names)
    
    def predict_batch(self, metrics_list):
        """вычислить скоры для батча"""
        return [self.predict(m) for m in metrics_list]
    
    def set_weights(self, weights):
        """установить веса"""
        self.weights = weights


class UniversalTextReRanker:
    """универсальный re-ranker для генераций текста"""
    
    def __init__(
        self,
        use_coverage=False,
        use_compression_ratio=False,
        use_extractive_coverage=False,
        use_fluency=False,
        use_grammar=False,
        use_length_simple=False,
        use_lexical_diversity=False,
        use_repetition_penalty=False,
        use_rouge_with_source=False,
        use_semantic_coherence=False,
        coverage_params=None,
        compression_params=None,
        length_params=None,
        weights=None,
        metric_selection='manual',
        min_correlation=0.15,
        device='cuda' if torch.cuda.is_available() else 'cpu',
        cache_dir='./reranker_cache'
    ):
        """
        args:
            use_*: флаги для включения метрик
            *_params: параметры для метрик
            weights: предустановленные веса
            metric_selection: 'manual', 'auto' или 'auto_all'
                - 'manual': используем только метрики с use_*=True
                - 'auto': из метрик с use_*=True отбираем по корреляции
                - 'auto_all': тестируем все метрики, отбираем по корреляции
            min_correlation: порог корреляции для auto режимов
            device: устройство для вычислений
            cache_dir: директория для кэша
        """
        self.use_coverage = use_coverage
        self.use_compression_ratio = use_compression_ratio
        self.use_extractive_coverage = use_extractive_coverage
        self.use_fluency = use_fluency
        self.use_grammar = use_grammar
        self.use_length_simple = use_length_simple
        self.use_lexical_diversity = use_lexical_diversity
        self.use_repetition_penalty = use_repetition_penalty
        self.use_rouge_with_source = use_rouge_with_source
        self.use_semantic_coherence = use_semantic_coherence
        
        self.coverage_params = coverage_params or {}
        self.compression_params = compression_params or {'optimal_ratio': 0.15, 'sigma': 0.05}
        self.length_params = length_params or {'target_min': 10, 'target_max': 20}
        
        if metric_selection not in ['manual', 'auto', 'auto_all']:
            raise ValueError(f"metric_selection must be 'manual', 'auto' or 'auto_all', got {metric_selection}")
        
        self.metric_selection = metric_selection
        self.min_correlation = min_correlation
        self.device = device
        self.cache_dir = Path(cache_dir)
        self.cache_dir.mkdir(exist_ok=True)
        
        self.metrics_computer = MetricsComputer(device=device)
        
        active_metrics = self._get_active_metrics()
        self.model = ReRankingModel(active_metrics, weights)
    
    def __getstate__(self):
        state = self.__dict__.copy()
        return state
    
    def __setstate__(self, state):
        self.__dict__.update(state)
    
    def _get_active_metrics(self):
        """получить список активных метрик"""
        metrics = []
        if self.use_coverage:
            metrics.append('coverage')
        if self.use_compression_ratio:
            metrics.append('compression_ratio')
        if self.use_extractive_coverage:
            metrics.append('extractive_coverage')
        if self.use_fluency:
            metrics.append('fluency')
        if self.use_grammar:
            metrics.append('grammar')
        if self.use_length_simple:
            metrics.append('length_simple')
        if self.use_lexical_diversity:
            metrics.append('lexical_diversity')
        if self.use_repetition_penalty:
            metrics.append('repetition_penalty')
        if self.use_rouge_with_source:
            metrics.append('rouge_with_source')
        if self.use_semantic_coherence:
            metrics.append('semantic_coherence')
        return metrics
    
    def _get_metric_params(self, metric_name):
        """получить параметры для метрики"""
        if metric_name == 'coverage':
            return self.coverage_params
        elif metric_name == 'compression_ratio':
            return self.compression_params
        elif metric_name == 'length_simple':
            return self.length_params
        else:
            return None
    
    def compute_batch(self, candidates, context=None):
        """
        вычислить все метрики для батча кандидатов
        
        args:
            candidates: list of str
            context: str или None
        
        returns:
            list of dict: [{metric_name: score}, ...]
        """
        if not candidates:
            return []
        
        all_scores = {}
        
        for metric_name in self.model.metric_names:
            params = self._get_metric_params(metric_name)
            all_scores[metric_name] = self.metrics_computer.compute_metric(
                metric_name, candidates, context, params
            )
        
        results = []
        for i in range(len(candidates)):
            candidate_scores = {
                metric: all_scores[metric][i] 
                for metric in all_scores
            }
            results.append(candidate_scores)
        
        return results
    
    def compute(self, text, context=None, return_individual=True, return_weighted=True):
        """вычислить метрики для одного текста"""
        batch_results = self.compute_batch([text], context)
        scores = batch_results[0]
        
        result = {}
        if return_individual:
            result['scores'] = scores
        
        if return_weighted:
            result['weighted_score'] = self.model.predict(scores)
        
        return result
    
    def rank_candidates(self, candidates, context=None):
        """
        ранжировать кандидатов
        
        returns:
            list of tuple: [(idx, weighted_score, individual_scores), ...]
        """
        if not candidates:
            return []
        
        batch_scores = self.compute_batch(candidates, context)
        
        results = []
        for idx, scores in enumerate(batch_scores):
            weighted_score = self.model.predict(scores)
            results.append((idx, weighted_score, scores))
        
        results.sort(key=lambda x: x[1], reverse=True)
        return results
    
    def get_best_candidate(self, candidates, context=None):
        """получить лучшего кандидата"""
        results = self.rank_candidates(candidates, context)
        if not results:
            return None
        best_idx = results[0][0]
        return candidates[best_idx]
    
    def fit(
        self,
        candidates_list,
        contexts,
        y_texts,
        metric='rouge2',
        cache_name=None,
        use_cache=True,
        max_iter=50,
        popsize=15,
        seed=42,
        n_workers=1,
        print_correlations=True
    ):
        """
        оптимизировать веса метрик
        
        args:
            candidates_list: list of list of str
            contexts: list
            y_texts: list of str
            metric: str - целевая метрика ('rouge2', 'rougeL')
        
        returns:
            dict, float: оптимальные веса и достигнутый скор
        """
        if len(candidates_list) != len(contexts) or len(candidates_list) != len(y_texts):
            raise ValueError(
                f"length mismatch: candidates_list={len(candidates_list)}, "
                f"contexts={len(contexts)}, y_texts={len(y_texts)}"
            )
        
        # определяем метрики для вычисления
        if self.metric_selection == 'auto_all':
            # вычисляем ВСЕ метрики
            metrics_to_compute = MetricsComputer.ALL_METRICS
            print(f"\nрежим auto_all: вычисляем все {len(metrics_to_compute)} метрик")
        else:
            # вычисляем только активные (use_*=True)
            metrics_to_compute = self._get_active_metrics()
            if self.metric_selection == 'auto':
                print(f"\nрежим auto: из {len(metrics_to_compute)} активных метрик отберём по корреляции >= {self.min_correlation}")
            else:
                print(f"\nрежим manual: используем {len(metrics_to_compute)} метрик")
        
        # предвычисление метрик
        precomputed_metrics = self._load_or_compute_metrics(
            candidates_list, contexts, cache_name, use_cache, metrics_to_compute
        )
        
        # предвычисление целевой метрики
        print(f"предвычисление целевой метрики ({metric})...")
        target_scores_cache = self._precompute_target_scores(
            candidates_list, y_texts, metric
        )
        
        metric_names = list(precomputed_metrics[0][0].keys())
        
        print(f"целевая метрика: {metric}")
        
        # вычисление корреляций
        correlations_dict = {}
        if print_correlations or self.metric_selection in ['auto', 'auto_all']:
            correlations_dict = self._compute_correlations(
                precomputed_metrics, target_scores_cache, metric_names
            )
        
        if print_correlations:
            self._print_correlations(correlations_dict)
        
        # автоматический отбор метрик
        if self.metric_selection in ['auto', 'auto_all']:
            selected_metrics = self._select_metrics_by_correlation(
                correlations_dict, self.min_correlation
            )
            
            if not selected_metrics:
                print(f"warning: ни одна метрика не прошла порог {self.min_correlation}")
                print(f"используем все {len(metric_names)} метрик")
                selected_metrics = metric_names
            else:
                dropped = set(metric_names) - set(selected_metrics)
                if dropped:
                    print(f"\nотброшены ({len(dropped)}):")
                    for m in sorted(dropped):
                        corr = correlations_dict[m]['mean']
                        print(f"  {m:25s}: корреляция {corr:+.3f}")
                
                print(f"\nотобрано: {len(selected_metrics)}/{len(metric_names)} метрик")
                print(f"метрики: {selected_metrics}")
            
            # фильтруем precomputed_metrics
            precomputed_metrics = [
                [{k: v for k, v in m.items() if k in selected_metrics} for m in batch]
                for batch in precomputed_metrics
            ]
            metric_names = selected_metrics
            
            # обновляем активные метрики в объекте (для auto_all)
            if self.metric_selection == 'auto_all':
                self._update_active_metrics(selected_metrics)
        
        n_metrics = len(metric_names)
        
        # оптимизация весов
        best_weights, best_score = self._optimize_weights(
            precomputed_metrics, target_scores_cache,
            metric_names, max_iter, popsize, seed, n_workers
        )
        
        # обновление модели
        weights_dict = {
            metric_names[i]: best_weights[i]
            for i in range(n_metrics)
        }
        
        # добавляем нулевые веса для неиспользуемых метрик
        for metric in MetricsComputer.ALL_METRICS:
            if metric not in weights_dict:
                weights_dict[metric] = 0.0
        
        # обновляем модель с новыми метриками
        self.model = ReRankingModel(metric_names, weights_dict)
        
        self._print_results(metric, best_score, weights_dict)
        
        return weights_dict, best_score
    
    def _update_active_metrics(self, selected_metrics):
        """обновить флаги use_* (для auto_all режима)"""
        for metric in MetricsComputer.ALL_METRICS:
            attr_name = f'use_{metric}'
            setattr(self, attr_name, metric in selected_metrics)
    
    def _compute_correlations(self, precomputed_metrics, target_scores_cache, metric_names):
        """вычислить корреляции метрик с целевой метрикой"""
        metric_correlations = {name: [] for name in metric_names}
        
        for i in range(len(precomputed_metrics)):
            target_values = target_scores_cache[i]
            
            for metric_name in metric_names:
                metric_values = [
                    precomputed_metrics[i][j][metric_name]
                    for j in range(len(precomputed_metrics[i]))
                ]
                
                if np.std(metric_values) > 1e-10 and np.std(target_values) > 1e-10:
                    corr, _ = spearmanr(metric_values, target_values)
                    metric_correlations[metric_name].append(corr)
        
        # усреднение
        avg_correlations = {}
        for metric_name in metric_names:
            if metric_correlations[metric_name]:
                avg_correlations[metric_name] = {
                    'mean': np.mean(metric_correlations[metric_name]),
                    'std': np.std(metric_correlations[metric_name])
                }
        
        return avg_correlations
    
    def _select_metrics_by_correlation(self, correlations_dict, min_correlation):
        """отобрать метрики по корреляции"""
        selected = []
        for metric_name, stats in correlations_dict.items():
            if abs(stats['mean']) >= min_correlation:
                selected.append(metric_name)
        return selected
    
    def _print_correlations(self, correlations_dict):
        """вывести корреляции"""
        print("\n" + "="*60)
        print("корреляция метрик с целевой метрикой (spearman)")
        print("="*60)
        
        correlations = [
            (name, stats['mean'], stats['std'])
            for name, stats in correlations_dict.items()
        ]
        correlations.sort(key=lambda x: abs(x[1]), reverse=True)
        
        for metric_name, avg_corr, std_corr in correlations:
            if abs(avg_corr) > 0.4:
                status = "сильная"
            elif abs(avg_corr) > 0.25:
                status = "средняя"
            elif abs(avg_corr) > 0.15:
                status = "слабая"
            else:
                status = "очень слабая"
            
            print(f"{status:15s} {metric_name:25s}: {avg_corr:+.3f} (±{std_corr:.3f})")
        
        print("="*60)
    
    def _print_results(self, metric, best_score, weights_dict):
        """вывести результаты калибровки"""
        print("\n" + "="*60)
        print("результаты калибровки")
        print("="*60)
        print(f"best {metric}: {best_score:.4f}")
        print(f"\nоптимальные веса:")
        
        sorted_weights = sorted(
            weights_dict.items(), 
            key=lambda x: abs(x[1]), 
            reverse=True
        )
        
        for name, weight in sorted_weights:
            if abs(weight) > 1e-6:
                print(f"  {name:25s}: {weight:+.4f}")
        
        print("="*60)
    
    def _load_or_compute_metrics(self, candidates_list, contexts, cache_name, use_cache, metrics_to_compute):
        """загрузить или вычислить метрики"""
        cache_path = None
        if cache_name and use_cache:
            # добавляем метрики в имя кэша для различения
            metrics_hash = hash(tuple(sorted(metrics_to_compute)))
            cache_path = self.cache_dir / f"{cache_name}_metrics_{metrics_hash}.pkl"
        
        if cache_path and cache_path.exists():
            print(f"загрузка метрик из кэша: {cache_path}")
            with open(cache_path, 'rb') as f:
                return pickle.load(f)
        
        print(f"предвычисление {len(metrics_to_compute)} метрик...")
        precomputed = []
        
        for candidates, context in tqdm(
            list(zip(candidates_list, contexts)), 
            desc="computing metrics"
        ):
            candidate_metrics = []
            for candidate in candidates:
                metrics_dict = {}
                for metric_name in metrics_to_compute:
                    params = self._get_metric_params(metric_name)
                    scores = self.metrics_computer.compute_metric(
                        metric_name, [candidate], context, params
                    )
                    metrics_dict[metric_name] = scores[0]
                candidate_metrics.append(metrics_dict)
            
            precomputed.append(candidate_metrics)
        
        if cache_path:
            with open(cache_path, 'wb') as f:
                pickle.dump(precomputed, f)
            print(f"метрики сохранены в кэш: {cache_path}")
        
        return precomputed
    
    def _precompute_target_scores(self, candidates_list, y_texts, metric='rouge2'):
        """предвычислить целевую метрику"""
        scorer = self.metrics_computer._load_rouge_scorer()
        target_scores = []
        
        for candidates, y_text in tqdm(
            zip(candidates_list, y_texts),
            total=len(candidates_list),
            desc="target metric"
        ):
            candidate_scores = []
            for candidate in candidates:
                scores = scorer.score(y_text, candidate)
                candidate_scores.append(scores[metric].fmeasure)
            
            target_scores.append(candidate_scores)
        
        return target_scores
    
    def _optimize_weights(
        self, 
        precomputed_metrics, 
        target_scores_cache, 
        metric_names,
        max_iter, 
        popsize, 
        seed, 
        n_workers
    ):
        """оптимизация весов"""
        n_metrics = len(metric_names)
        
        # сохраняем для objective function
        self._opt_precomputed_metrics = precomputed_metrics
        self._opt_target_scores_cache = target_scores_cache
        self._opt_metric_names = metric_names
        self._opt_n_metrics = n_metrics
        
        print(f"\nоптимизация весов (max_iter={max_iter}, popsize={popsize}, workers={n_workers})...")
        
        result = differential_evolution(
            self._objective_function,
            bounds=[(-1, 1)] * n_metrics,
            strategy='best1bin',
            maxiter=max_iter,
            popsize=popsize,
            tol=0.001,
            mutation=(0.5, 1.5),
            recombination=0.7,
            seed=seed,
            workers=n_workers,
            updating='deferred' if n_workers > 1 else 'immediate',
            polish=True,
            disp=True
        )
        
        # очистка
        del self._opt_precomputed_metrics
        del self._opt_target_scores_cache
        del self._opt_metric_names
        del self._opt_n_metrics
        
        weights_raw = result.x
        weights_sum = np.sum(np.abs(weights_raw))
        
        if weights_sum < 1e-10:
            print(f"warning: все веса близки к нулю")
            weights_normalized = np.ones(n_metrics) / n_metrics
        else:
            weights_normalized = weights_raw / weights_sum
        
        best_score = -result.fun
        return weights_normalized, best_score
    
    def _objective_function(self, weights):
        """функция для оптимизации"""
        total_score = 0.0
        
        for metrics_list, target_scores in zip(
            self._opt_precomputed_metrics,
            self._opt_target_scores_cache
        ):
            # вычисляем взвешенный скор
            weighted_scores = [
                sum(
                    weights[i] * metrics_dict.get(self._opt_metric_names[i], 0)
                    for i in range(self._opt_n_metrics)
                )
                for metrics_dict in metrics_list
            ]
            
            # выбираем лучшего
            best_idx = np.argmax(weighted_scores)
            total_score += target_scores[best_idx]
        
        avg_score = total_score / len(self._opt_precomputed_metrics)
        return -avg_score
    
    def save(self, path):
        """сохранить re-ranker"""
        with open(path, 'wb') as f:
            pickle.dump(self, f)
        print(f"re-ranker сохранен: {path}")
    
    @staticmethod
    def load(path):
        """загрузить re-ranker"""
        with open(path, 'rb') as f:
            ranker = pickle.load(f)
        print(f"re-ranker загружен: {path}")
        return ranker

In [143]:
def predict_with_reranker(promts):
    results = []
    for promt in promts:
        generated_texts = generate_response(
            trainer.model.half(), 
            tokenizer,
            [promt],
            num_return_sequences=10,
            num_beams=4,
            temperature=0.8,
            top_k=10,
            top_p=0.9,
            max_new_tokens=40
        )[0]
        result = ranker.get_best_candidate(generated_texts, promt)
        results.append(result)
    return results

def evaluate_model_with_reranker(model, tokenizer, val_data, batch_size=8):
    predictions = []

    for i in tqdm(range(0, len(val_data), batch_size)):
        batch = [val_data[j] for j in range(i, min(i + batch_size, len(val_data)))]
        batch_predictions = predict_with_reranker([template_processing(item['question']) for item in batch])
        predictions.extend(batch_predictions)

    references = [item['answer'] for item in val_data]

    rouge = evaluate.load('rouge')
    final_scores = rouge.compute(predictions=predictions, references=references)

    return final_scores, predictions

def get_ranker_train_data(indexes):
    dev_candidates = []
    dev_x_texts = []
    dev_y_texts = []

    for sample in tqdm(test_data.select(indexes)):
        candidates = generate_response(
            trainer.model.half(), tokenizer, [template_processing(sample['question'])],
            num_return_sequences=10,
            num_beams=2,
            temperature=0.8,
            top_k=10,
            top_p=0.9,
            max_new_tokens=40
        )[0]

        dev_candidates.append(candidates)
        dev_x_texts.append(sample['question'])
        dev_y_texts.append(sample['answer'])

    return dev_candidates, dev_x_texts, dev_y_texts

In [166]:
# ranker = UniversalTextReRanker(
#     use_coverage=True,
#     use_fluency=True,
#     # use_grammar=True,
#     use_length_simple=True,
#     use_repetition_penalty=True,
#     length_params={'target_min': 1, 'target_max': 15}
# )

ranker = UniversalTextReRanker(
    metric_selection='auto_all',
    min_correlation=0.15
)

# dev_candidates, dev_x_texts, dev_y_texts = get_ranker_train_data(list(range(200)))

optimal_weights, best_score = ranker.fit(
    candidates_list=dev_candidates,
    contexts=dev_x_texts,
    y_texts=dev_y_texts,
    metric='rouge2',
    cache_name='commongen_dev_v9',
    max_iter=50,
    popsize=30,
    print_correlations=True
)


режим auto_all: вычисляем все 10 метрик
предвычисление 10 метрик...


computing metrics:   0%|          | 0/200 [00:00<?, ?it/s]

загрузка модели для fluency (gpt-2)...
метрики сохранены в кэш: reranker_cache/commongen_dev_v9_metrics_-4645364536204692210.pkl
предвычисление целевой метрики (rouge2)...


target metric:   0%|          | 0/200 [00:00<?, ?it/s]

целевая метрика: rouge2

корреляция метрик с целевой метрикой (spearman)
сильная         compression_ratio        : -0.596 (±0.523)
сильная         coverage                 : -0.527 (±0.406)
сильная         rouge_with_source        : -0.504 (±0.425)
сильная         semantic_coherence       : +0.444 (±0.371)
сильная         repetition_penalty       : +0.424 (±0.403)
средняя         fluency                  : -0.334 (±0.446)
слабая          lexical_diversity        : -0.245 (±0.445)
очень слабая    length_simple            : -0.077 (±0.620)
очень слабая    extractive_coverage      : +0.050 (±0.465)
очень слабая    grammar                  : +0.002 (±0.454)

отброшены (3):
  extractive_coverage      : корреляция +0.050
  grammar                  : корреляция +0.002
  length_simple            : корреляция -0.077

отобрано: 7/10 метрик
метрики: ['coverage', 'compression_ratio', 'fluency', 'lexical_diversity', 'repetition_penalty', 'rouge_with_source', 'semantic_coherence']

оптимизация весо

In [167]:
def evaluate_on_cached_candidates(dev_candidates, dev_x_texts, dev_y_texts):
    predictions = []

    for candidates, x_text in tqdm(list(zip(dev_candidates, dev_x_texts))):
        best = ranker.get_best_candidate(candidates, x_text)
        predictions.append(best)

    rouge = evaluate.load('rouge')
    scores = rouge.compute(predictions=predictions, references=dev_y_texts)

    return scores, predictions

scores, preds = evaluate_on_cached_candidates(dev_candidates, dev_x_texts, dev_y_texts)
print(scores)

  0%|          | 0/200 [00:00<?, ?it/s]

{'rouge1': 0.607136387972391, 'rouge2': 0.40260392064501627, 'rougeL': 0.6058775773495776, 'rougeLsum': 0.604456859396507}


In [ ]:
# проверить catboostranker

scores, preds = evaluate_model_with_reranker(trainer.model.half(), tokenizer, test_data.select(range(200)), batch_size=1)

print(scores)
for pred in preds:
    print('\n########################\n')
    print(pred)

  0%|          | 0/200 [00:00<?, ?it/s]